<a href="https://colab.research.google.com/github/basugautam/Reproducibility-Challenge-Project/blob/Architecture-Files/9_Dynamic_Constraint_Adaptation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Required Libraries
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader

# 🔹 Step 1: Load and Preprocess Dataset
dataset_path = r"C:\LCTSF\Dataset\timeseries_dataset.csv"
df = pd.read_csv(dataset_path)

# Extract time-series values
data = df['value'].values.reshape(-1, 1)

# Normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# 🔹 Step 2: Create Time-Series Sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length, 0])
        y.append(data[i + seq_length, 0])
    return np.array(X), np.array(y)

sequence_length = 50
X, y = create_sequences(data_scaled, sequence_length)

# Split into Training & Testing Sets
split_idx = int(0.8 * len(X))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# Convert data to PyTorch tensors
X_train_torch = torch.tensor(X_train, dtype=torch.float32).unsqueeze(-1)  # Add feature dimension
y_train_torch = torch.tensor(y_train, dtype=torch.float32).unsqueeze(-1)

X_test_torch = torch.tensor(X_test, dtype=torch.float32).unsqueeze(-1)
y_test_torch = torch.tensor(y_test, dtype=torch.float32).unsqueeze(-1)

# Create PyTorch Dataset
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Define DataLoaders
batch_size = 32
train_loader = DataLoader(TimeSeriesDataset(X_train_torch, y_train_torch), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TimeSeriesDataset(X_test_torch, y_test_torch), batch_size=batch_size, shuffle=False)

# 🔹 Step 3: Define Transformer Model with Dynamic Constraints
class TransformerForecasting(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, dropout=0.1):
        super(TransformerForecasting, self).__init__()
        self.embedding = nn.Linear(input_dim, model_dim)  # Project input to model dimension
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(model_dim, 1)  # Final prediction layer
        self.constraint_factor = torch.nn.Parameter(torch.tensor(1.0))  # Dynamic constraint parameter

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer_encoder(x)
        x = self.fc(x[:, -1, :])  # Use last time step for prediction
        return x

# Instantiate Transformer Model
input_dim = 1
model_dim = 64
num_heads = 4
num_layers = 2

transformer_model = TransformerForecasting(input_dim, model_dim, num_heads, num_layers)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(transformer_model.parameters(), lr=0.001)

# 🔹 Step 4: Define Dynamic Constraint Adaptation Function
def dynamic_constraint(loss, constraint_factor, threshold=0.05):
    """
    Adjusts constraints dynamically based on training performance.

    - If the loss is below a certain threshold, reduce constraints to allow flexibility.
    - If the loss is above the threshold, increase constraints to enforce better accuracy.
    """
    with torch.no_grad():
        if loss < threshold:
            constraint_factor *= 0.95  # Reduce constraint (allow more flexibility)
        else:
            constraint_factor *= 1.05  # Tighten constraint (force better accuracy)
    return constraint_factor

# 🔹 Step 5: Train Model with Dynamic Constraint Adaptation
def train_model(model, train_loader, criterion, optimizer, epochs=50):
    model.train()
    constraint_factor = torch.tensor(1.0)  # Initial constraint factor

    for epoch in range(epochs):
        total_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch) * constraint_factor  # Apply dynamic constraint
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Update constraint dynamically based on loss behavior
        constraint_factor = dynamic_constraint(total_loss / len(train_loader), constraint_factor)

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.6f}, Constraint Factor: {constraint_factor:.4f}")

train_model(transformer_model, train_loader, criterion, optimizer, epochs=50)

# 🔹 Step 6: Evaluate Model Performance
def evaluate_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    predictions, actuals = [], []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            total_loss += loss.item()
            predictions.append(y_pred.numpy())
            actuals.append(y_batch.numpy())

    return np.concatenate(predictions), np.concatenate(actuals), total_loss / len(test_loader)

y_pred_test, y_test_actual, test_loss = evaluate_model(transformer_model, test_loader, criterion)

# Rescale Predictions to Original Scale
y_test_actual_rescaled = scaler.inverse_transform(y_test_actual)
y_pred_test_rescaled = scaler.inverse_transform(y_pred_test)

# 🔹 Step 7: Visualizing Forecasting Performance
plt.figure(figsize=(12, 6))
plt.plot(y_test_actual_rescaled, label="Actual", color="blue")
plt.plot(y_pred_test_rescaled, label="Predicted", color="red", linestyle="dashed")
plt.title("Transformer Model - Dynamic Constraint Adaptation Forecasting")
plt.xlabel("Time Steps")
plt.ylabel("Value")
plt.legend()
plt.show()

print(f"🔹 Test Loss (MSE): {test_loss:.6f}")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\LCTSF\\Dataset\\timeseries_dataset.csv'